## Цели проекта 

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.
Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

## Исследование данных

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime
pd.set_option('display.max_columns', None)
pd.options.display.max_colwidth = 150

In [2]:
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})


### Books

In [3]:
query = '''
            SELECT * FROM books
        '''
books = pd.io.sql.read_sql(query, con = engine)

In [4]:
books.head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope #1),322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before Columbus,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


### Authors

In [5]:
query = '''
            SELECT * FROM authors
        '''
authors = pd.io.sql.read_sql(query, con = engine)

In [6]:
authors.head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


### Publishers

In [8]:
query = '''
            SELECT * FROM publishers
        '''
publishers = pd.io.sql.read_sql(query, con = engine)

In [9]:
publishers.head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


### Ratings

In [10]:
query = '''
            SELECT * FROM ratings
        '''
ratings = pd.io.sql.read_sql(query, con = engine)

In [11]:
ratings.head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


### Reviews

In [12]:
query = '''
            SELECT * FROM reviews
        '''
reviews = pd.io.sql.read_sql(query, con = engine)

In [13]:
reviews.head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. Over provide race technology continue these.
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Among admit investment argue security.
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but person sport treatment industry. Kitchen decision deep the. Social party body the.
3,4,3,johnsonamanda,Finally month interesting blue could nature cultural bit. Prepare beat finish grow that smile teach. Dream me play near.
4,5,3,scotttamara,Nation purpose heavy give wait song will. List dinner another whole positive radio fast. Music staff many green.


## Количество книг, вышедших после 1 января 2000 года

In [16]:
<div class="alert alert-block alert-success">✔️
    

__Комментарий от тимлида №1__

Расчёт верный
</div>counter = '''
                SELECT 
                COUNT(book_id) AS counter_book
                FROM
                    books
                WHERE publication_date > '2000-01-01';
             ''' 
counter_book = pd.io.sql.read_sql(counter, con = engine)
counter_book

,counter_book
0,819


После 1 января 2000 года вышло 819 книг

## Количество обзоров и средняя оценка по каждой книге 

In [29]:
cnt_avg = '''                              
                    SELECT
                        books.title AS book_title,
                        books.book_id AS book_id,
                        AVG(ratings.rating) AS avg,
                        COUNT(DISTINCT reviews.review_id) AS count
                     FROM books

                    INNER JOIN ratings ON ratings.book_id = books.book_id
                    LEFT JOIN reviews ON reviews.book_id = books.book_id
                    

                    GROUP BY
                        books.title,
                        books.book_id

                    ORDER BY
                        count DESC;
                ''' 

count_avg = pd.io.sql.read_sql(cnt_avg, con = engine)
count_avg

,book_title,book_id,avg,count
0,Twilight (Twilight #1),948,3.662500,7
1,Water for Elephants,963,3.977273,6
2,The Glass Castle,734,4.206897,6
3,Harry Potter and the Prisoner of Azkaban (Harry Potter #3),302,4.414634,6
4,The Curious Incident of the Dog in the Night-Time,695,4.081081,6
...,...,...,...,...
995,Anne Rice's The Vampire Lestat: A Graphic Novel,83,3.666667,0
996,The Natural Way to Draw,808,3.000000,0
997,The Cat in the Hat and Other Dr. Seuss Favorites,672,5.000000,0
998,Essential Tales and Poems,221,4.000000,0


Количество обзоров по каждой книге варьируется от 0 до 7

## Издательство, которое выпустило наибольшее число книг толще 50 страниц

In [31]:
query1 = '''
SELECT
    publishers.publisher,
    COUNT(books.book_id) AS count_books
FROM 
    books
INNER JOIN publishers ON books.publisher_id = publishers.publisher_id
WHERE
    books.num_pages > 50
GROUP BY
    publishers.publisher
ORDER BY
    count_books DESC
    LIMIT 1
'''

request1 = pd.io.sql.read_sql(query1, con = engine)
request1

,publisher,count_books
0,Penguin Books,42


Издательство, которое выпустило наибольшее число книг толще 50 страниц - это Penguin Books

## Автор с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками

In [32]:
query2 = '''
                        SELECT
                           authors.author_id AS id,
                           authors.author AS author,
                           AVG(ratings.rating) AS avg_rating,
                           COUNT(ratings.rating) AS count_rating
                        FROM
                           authors
                        INNER JOIN books ON books.author_id = authors.author_id
                        INNER JOIN ratings ON ratings.book_id = books.book_id
                        GROUP BY
                           id,
                           author
                        HAVING 
                           COUNT(ratings.rating) >= 50 
                           
                        ORDER BY
                           count_rating DESC
                        LIMIT 1
                        ;
                ''' 

request2 = pd.io.sql.read_sql(query2, con = engine)
request2

,id,author,avg_rating,count_rating
0,236,J.K. Rowling/Mary GrandPré,4.288462,312


Автор с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками - J.K. Rowling

Cреднее количество обзоров от пользователей, которые поставили больше 48 оценок - 24